# 径向基函数网络/RBF网络

RBF网络时具有隐藏层的前向网络，隐藏层以RBF作为激活函数。不同于一般的BP网络，RBF网络具有局部逼近的特性。并且Poggio和Girosi已经证明RBF网络时连续函数的最佳逼近。

BP网络和RBF网络的区别见下图

![BP网络和RBF网络对比](![moons](https://raw.githubusercontent.com/koolo233/NeuralNetworks/main/images/bp_rbf.jpg "segment"))

## RBF网络的正向计算

### 定义RBF函数

$$
\begin{equation}
    \phi(\bm{x}, \bm{c}) = \phi(||\bm{x}-\bm{c}||)
\end{equation}
$$
上式中$c$为中心坐标，$\phi$为高斯核函数

* 以高斯核函数和欧氏距离定义的RBF函数
$$
\begin{equation}
    \varphi(\bm{x}) = exp(-\frac{1}{2}\sum_{i=1}^{k}\frac{(x_i-c_i)^2}{\sigma_i^2})
\end{equation}
$$

上式中，输入维度为$k$，$c_i$和$\sigma_i$分别为第$i$个维度的中心坐标和标准化参数

若所有维度均采用相同的标准化参数，则上式可简化为
$$
\begin{equation}
    \varphi(\bm{x}) = exp(-\frac{1}{2\sigma^2}||\bm{x}-\bm{c}||^2)
\end{equation}
$$

定义
$$
\begin{equation}
    \beta = \frac{1}{2\sigma^2}, \  \beta \geq0
\end{equation}
$$

有
$$
\begin{equation}
    \varphi(\bm{x}) = exp(-\beta||\bm{x}-\bm{c}||^2)
\end{equation}
$$


### 从输入层到隐藏层

假设隐藏层有n个神经元，则对于每一个神经元有
$$
\begin{equation}
    \varphi_i = exp(-(\bm{x}-\bm{c_i})^Tdiag(\bm{x}-\bm{c_i})\bm{\beta_i}), \ i = 1, 2, 3, \cdots, n
\end{equation}
$$

### 从隐藏层到输出层
假设输出层有o个输出单元，则对于输出层有
$$
\begin{equation}
    \begin{split}
        \bm{y} 
        &= \bm{W}\bm{\varphi} \\
        &= 
        \left[
            \begin{array}{cc}
                \bm{w_1} \\
                \bm{w_2} \\
                \vdots \\
                \bm{w_o} \\
            \end{array}
        \right]
        \left[
            \begin{array}{cc}
                \varphi_1 \\
                \varphi_2 \\
                \vdots \\
                \varphi_n
            \end{array}
        \right]
    \end{split}
\end{equation}
$$

## RBF网络的学习算法

RBF网络待学习参数如下：
1. 高斯RBF函数的中心坐标$\bm{c}$和参数$\beta$
2. 隐藏层到输出层的权重$\bm{W}$

### 自组织选取中心学习算法
1. 无监督学习过程：求解高斯RBF函数相关参数
2. 有监督学习过程：求解隐藏层到输出层的权重

对于第一步，可以使用聚类相关方法来进行。即对于n个RBF单元，通过对输入样本聚类为n类来获得相应的中心坐标以及散度参数。


在完成第一步后，第二步为确定隐藏层到输出层的权重。这一步为线性映射，因此可以使用各种线性优化算法来求得最优解，例如最小二乘法等。

### 监督学习
对于RBF网络，其可以使用类似于一般BP算法中的梯度下降法对网络参数进行求解。在本文件中主要是实现这一优化方法。

## RBF网络的监督学习

* 定义损失
$$
\begin{equation}
    \mathcal{L(\bm{c}, \bm{\beta}, \bm{W})} = \frac{1}{M}\sum_{i=1}^M||\bm{\bar{y}}_i - \bm{y}_i||^2_2
\end{equation}
$$
其中$\bm{\bar{y}}_i$为第$i$个样本的输出，$\bm{y}_i$为第$i$个样本的标签，$M$为训练集的样本总数

* 定义优化函数
$$
\begin{equation}
    \min\limits_{\bm{c}, \bm{\beta}, \bm{W}}{\mathcal{L}(\bm{c}, \bm{\beta}, \bm{W})}=\frac{1}{M}\sum_{i=1}^M||\bm{\bar{y}}_i - \bm{y}_i||^2_2
\end{equation}
$$
* 误差反向传播(单样本)
    * $\bm{W}$
    $$
    \begin{equation}
        \begin{split}
            \frac{\partial \mathcal{L}}{\partial \bm{W}} 
            &= \frac{\partial \mathcal{L}}{\partial \bm{\bar{y}}} \frac{\partial \bm{\bar{y}}}{\partial \bm{W}} \\
            &= 2(\bm{\bar{y}}-\bm{y})\bm{\varphi}^T
        \end{split}
    \end{equation}
    $$
    * $\bm{\varphi}$
    $$
    \begin{equation}
        \begin{split}
            \frac{\partial \mathcal{L}}{\partial \bm{\varphi}}
            &= \frac{\partial \mathcal{L}}{\partial \bm{\bar{y}}} \frac{\partial \bm{\bar{y}}}{\partial \bm{\varphi}} \\ 
            &= 2(\bm{\bar{y}}-\bm{y})^T \bm{W}
        \end{split}
    \end{equation}
    $$
    * $\bm{c}$
    对于每一个隐藏单元的中心参数$\bm{c_i}$有
    $$
    \begin{equation}
        \begin{split}
            \frac{\partial \mathcal{L}}{\partial \bm{c_i}}
            &= \frac{\partial \mathcal{L}}{\partial \bm{\varphi_i}} \frac{\partial \bm{\varphi_i}}{\partial \bm{c_i}} \\ 
            &= (\sum_{j=1}^o2(\bar{y}_j-y_j)\bm{W}_{ji})(\varphi_i)(2\bm{\beta_i}^Tdiag(\bm{x}-\bm{c_i }))
        \end{split}, i = 1, 2, \cdots, n
    \end{equation}
    $$
    * $\bm{\beta}$
    对于每一个隐藏单元的参数$\bm{\beta_i}$有
    $$
    \begin{equation}
        \begin{split}
            \frac{\partial \mathcal{L}}{\partial \bm{\beta_i}}
            &= \frac{\partial \mathcal{L}}{\partial \bm{\varphi_i}} \frac{\partial \bm{\varphi_i}}{\partial \bm{\beta_i}} \\
            &= (\sum_{j=1}^o2(\bar{y}_j-y_j)\bm{W}_{ji}) (\varphi_i)( -(\bm{x}-\bm{c_i})^Tdiag(\bm{x}-\bm{c_i}))
        \end{split}, i = 1, 2, \cdots, n
    \end{equation}
    $$

* 更新参数
    * $\bm{W}$
    $$
    \begin{equation}
        \bm{W}_{t+1} = \bm{W}_t - \eta\frac{\partial \mathcal{L}}{\partial \bm{W}}
    \end{equation}
    $$
    * $\bm{c}$
    对于每一个隐藏单元的中心参数$\bm{c_i}$有
    $$
    \begin{equation}
        \bm{c_i}_{t+1} = {c_{i}}_t - \eta (\frac{\partial \mathcal{L}}{\partial \bm{c_i}})^T
    \end{equation}
    $$
    * $\bm{\beta}$
    对于每一个隐藏单元的参数$\bm{\beta_i}$有
    $$
    \begin{equation}
        \bm{\beta_i}_{t+1} = {\beta_{i}}_t - \eta (\frac{\partial \mathcal{L}}{\partial \bm{\beta_i}})^T
    \end{equation}
    $$